# Project 3 ANN

Ashton Cole

AVC687

COE 379L: Software Design for Responsive Intelligent Systems

## Description

This notebook builds a densely-connected Artificial Neural Network.

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

2024-04-09 23:13:24.090570: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 23:13:24.134365: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 23:13:24.134426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 23:13:24.137046: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-09 23:13:24.147673: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 23:13:24.148671: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Preprocessing

In [2]:
validation_split = 0.2 # Validation split
batch_size = 32
seed = 123

image_height = 128
image_width = 128

train_dir = 'data/split/train'
test_dir = 'data/split/test'

rescale = Rescaling(scale=1.0/255)

data_train, data_validation = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=validation_split,
    subset='both',
    seed=seed,
    image_size=(image_height, image_width),
    batch_size=batch_size)
data_test = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    seed=seed,
    image_size=(image_height, image_width),
    batch_size=batch_size)

data_train_scaled = data_train.map(lambda image, label: (rescale(image), label))
data_validation_scaled = data_validation.map(lambda image, label: (rescale(image), label))
data_test_scaled = data_test.map(lambda image, label: (rescale(image), label))

Found 17057 files belonging to 2 classes.
Using 13646 files for training.
Using 3411 files for validation.
Found 4265 files belonging to 2 classes.


## Model Construction

In [3]:
image_length = image_height * image_width * 3

model = Sequential()
# F1
model.add(Flatten(input_shape=(image_height, image_width, 3)))
# F2
model.add(Dense(128, activation='relu'))
# F3
model.add(Dense(128, activation='relu'))
# Output
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 49152)             0         
                                                                 
 dense (Dense)               (None, 128)               6291584   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 6308354 (24.06 MB)
Trainable params: 6308354 (24.06 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Model Compilation and Fitting

In [4]:
optimizer='adam'
loss='sparse_categorical_crossentropy'
metrics=['accuracy']

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics)

model.fit(
    x=data_train_scaled,
    epochs=5,
    validation_data=data_validation_scaled,
    verbose=2,
    batch_size=batch_size)

Epoch 1/5
427/427 - 16s - loss: 0.8669 - accuracy: 0.6453 - val_loss: 0.6387 - val_accuracy: 0.6775 - 16s/epoch - 37ms/step
Epoch 2/5
427/427 - 18s - loss: 0.5976 - accuracy: 0.6970 - val_loss: 0.6683 - val_accuracy: 0.5731 - 18s/epoch - 43ms/step
Epoch 3/5
427/427 - 20s - loss: 0.6083 - accuracy: 0.6784 - val_loss: 0.5733 - val_accuracy: 0.6681 - 20s/epoch - 46ms/step
Epoch 4/5
427/427 - 21s - loss: 0.5997 - accuracy: 0.6637 - val_loss: 0.6606 - val_accuracy: 0.6681 - 21s/epoch - 50ms/step
Epoch 5/5
427/427 - 23s - loss: 0.6421 - accuracy: 0.6637 - val_loss: 0.6358 - val_accuracy: 0.6681 - 23s/epoch - 55ms/step


## Model Testing

In [5]:
test_loss, test_accuracy = model.evaluate(data_test_scaled, verbose=0)
test_accuracy

0.6644783020019531